In [ ]:
import accessvis

In [ ]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import os
import pandas as pd
import os
import xarray as xr
import lavavu

from tqdm import tqdm
from scipy.ndimage import gaussian_filter

path="/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/3hr/pr/gn/latest"
file="pr_3hr_ACCESS-ESM1-5_historical_r1i1p1f1_gn_201001010130-201412312230.nc"

In [ ]:
ds=xr.open_dataset(f"{path}/{file}")
ds["pr"][0]

In [ ]:
def get_interpolated(data, colourmap, blur=2.5):
    lat = data['lat'].values
    lon = data['lon'].values
    newlon = np.linspace(lon[0], lon[-1], 2048)
    newlat = np.linspace(lat[0], lat[-1], 1024)

    interpolated = data.interp(lon=newlon, lat=newlat, method="linear")
    if blur > 0:
        # interpolated_array=interpolated.to_dataarray()
        smoothed = gaussian_filter(interpolated, sigma=blur)
        #smoothed = median_filter(interpolated, size=blur)
    else:
        #No blur
        smoothed = interpolated

    #Convert to image, use raw interpolated data as opacity map for transparency
    rgba = accessvis.array_to_rgba(smoothed, flip=True, colourmap=colourmap, opacitymap=True)
    return rgba

In [ ]:
ds_exa=ds.isel(time=2)["pr"]
colours = get_interpolated(ds_exa, 'turbo')
plt.imshow(colours)

### Video

In [ ]:
filename = 'el_test_pr_3hr.mp4'
lv = accessvis.plot_earth(texture='bluemarble', background="white", vertical_exaggeration=20)
lv.rotation(15.0, -190.0, 0.0) #Rotate to Australia
lv.set_properties(diffuse=0.8, ambient=0.1, specular=0.35, shininess=0.03, light=[1,1,0.98]) # make pretty
lv.brightness_contrast_saturation(0.5, 0.5, 0.65)
date = accessvis.TextWidget(lv=lv, text_colour = 'black', scale=0.05, offset=(1,0))
with lv.video(filename=f'el_test_pr_3hr.mp4',fps=60, quality=3, resolution=(600,600)) as v:
    for i in tqdm(range(len(ds["pr"]))):
        formatted_date = pd.to_datetime(ds["pr"][i].time.data).strftime('%Y/%m/%d')
        date.update_widget(text=f"{formatted_date}")
        colours = get_interpolated(ds["pr"][i], 'gist_ncar')
        accessvis.update_earth_values(lv, dataMode=0, data=colours)
        lv.render()